# Contract Review
___

这个示例用于演示，在一个通用的组织架构中，agent 间相互配合，自行完成合同审核任务

具体的配置过程推荐阅读 [Complete Usage](./complete_usage.ipynb) 笔记

## 安装框架
___

AutoGan requires Python version >= 3.8. It can be installed from pip:

In [ ]:
%%capture --no-stderr
%pip install --upgrade pyautogan

In [ ]:
import autogan

## LLM 配置
___

In [ ]:
# 通过 dict_from_json 函数读取 LLM 配置
llm_config_dict = autogan.dict_from_json("LLM_CONFIG")

## 构建 Agents
___

### 数字化身

人类的数字化身，用来与其他 agent 协同工作。

In [ ]:
human = autogan.HumanAgent("客户", duty="请帮助我完成业务")

### 客户经理

In [ ]:
cust_manager = autogan.UniversalAgent("客户经理", duty="负责接待客户", work_flow="""
1. 你是一个富有经验的客户经理，请尽一切可能满足客户的合法需求。

2. 如果没有正确答案，请明确告知，而不是给出错误结果""")

## 审核员

In [ ]:
Auditor = autogan.UniversalAgent("审核员", duty="请将待审核合同的文件名发送给我，我将返回审核结果", work_flow="""
收到合同后，请站在银行的角度，从以下方面审核发送给你的合同：
1. 合同条款的完整性和合法性：审核合同中的条款是否完整，确保没有遗漏重要的法律要求，比如付款条件、交货期限、违约责任等。

2. 风险评估：分析和评估合同执行过程中可能出现的风险，例如供货延迟、质量问题、市场风险等。

3. 合规性检查：确保合同遵守了所有相关的法律法规和内部政策，特别是关于反洗钱和反腐败的规定。

4. 付款条件和流程：审核合同中的付款条件，确保它们是公平且可行的，并符合银行的内部流程。""")

### 其他预置 agent

1. 搜索专家

In [ ]:
search_config_dict = autogan.dict_from_json("SEARCH_CONFIG")

In [ ]:
search_exp = autogan.ToolAgentSearch(search_config_dict, name="搜索专家")

2. 秘书

In [ ]:
mail_config_dict = autogan.dict_from_json("MAIL_CONFIG")

In [ ]:
secretary = autogan.ToolAgentMail(mail_config_dict, "秘书")

3. 文件助手

In [ ]:
file_exp = autogan.ToolAgentFile(name="文件助手")

4. 程序员

In [ ]:
coder = autogan.UniversalAgent("程序员", duty="我可以编写 python 代码并执行", work_flow="""
1. 我希望你是一个有经验的Python程序员，将接收到的需求用代码来实现。不用管你自己是否有能力执行代码，因为 测试员 可以帮你执行。

2. 你的代码需要先 @测试员，并将代码使用 ``` 符号封装，他会回复执行结果，例如：
@测试员 请帮我执行代码
```python
Your code
```""")

In [ ]:
test_staff = autogan.ToolAgentCodeExecution("测试员")

## 构建组织架构
___

考虑到并非所有 agent 之间都需要建立协作关系，且过多无用的协作关系会影响到 agent 决策的准确性。
因此本框架引入了组织架构的概念，让需要相互协作的 agent 组成独立的数字部门，以更加专注于解决特定领域的问题。

创建组织架构的方式非常简单，只需要定义一个存储 agent 对象的多维列表。

In [ ]:
org_structure = [human, cust_manager, Auditor, search_exp, secretary, file_exp, [coder, test_staff]]

每个 list 相当于一个部门，各部门的第一个 agent 为部门的 leader，负责与上级部门的 agent 进行沟通。
因此 leader 的 duty 属性应当概述整个部门的职测与能力。

## 开始会话
___

与 agent 的初次对话，需要在发送的消息中添加 task_tag 符号，用于创建任务，task_tag 的默认值为 "/task"。

后续对话中如果使用 task_tag 符号，则是对当前任务发表意见或看法。

In [ ]:
autogan.AgentSwitch(org_structure, "hello", human, llm_config_dict);